In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time
import csv
import pandas as pa
%matplotlib inline

In [3]:
#パラメータの定義
nPh = 10 #光子数

####  モンテカルロ用パラメーター  #####
###パラメーターの設定###
#それぞれの大きさはcmで記載

#皮膚
g_skin = 0.9   #異方性係数
ms_skin = 180  #散乱係数
ma_skin = 0.13  #吸収係数
n_skin = 1.4   #屈折率
d_skin = [0.05, 0.1, 0.15, 0.2]   #厚さ

#海綿骨
g_bone = 0.93
ms_bone = [50, 100, 150, 200, 250, 300, 350, 400]
ma_bone = 0.03
n_bone = 1.4
d_bone = 0.03

#皮質骨
g_cbone = 0.9
ms_cbone = 1800
ma_cbone = 0.03
n_cbone = 1.4
d_cbone = 10

####  光路計算用パラメーター  #####
#捜査範囲（ここだけ ｍｍ 記述になります）
start = -10 #どこから計算をスタートさせるか
end = 90    #どこまで計算させるか
split = 1     #何ｍｍごとに分割するかを入力


#パラメータの設定(cm)
#レンズ１
outerD_1 = 5    #外径
efl_1 = 10      #焦点距離
bfl_1 = 9.341   #後焦点距離
ct_1 = 1        #中心厚
et_1 = 0.3553   #コバ厚
r_1 = 5.168     #曲率半径
n_1 = 1.517     #屈折率

#レンズ２
outerD_2 = 5
efl_2 = 5
bfl_2 = 4.328
ct_2 = 1.2
et_2 = 0.301
r_2 = 3.924
n_2 = 1.758

#スリット
slit_outerD = 5  #外径
slitD = 2        #スリット径
width = 0.2    #スリット間隙の大きさ
thickness = 0.3  #厚さ

#フォトダイオード
d_pd = 0.6       #受光面直径

print("Parameter setting is complete")

Parameter setting is complete


In [2]:
####  各クラス ####

####################　モンテカルロ用クラス　####################
class Photon(object):
    def __init__ (self):
        self.w = 1                        #光子のエネルギー
        self._wMin = 0.0001               #光子の最小エネルギー
        self.u = np.array([0,0,1])        #光子の方向ベクトル
        self.coor = np.array([0,0,0])  #光子の位置座標
        self.nPh = 0
        
    #for Normal incidence
    def nomalIncidence(self,n):
        Rsp = ((1-n)**2)/((1+n)**2)
        return 0, 0, 1,Rsp
    
    #for Oblique incidence   
    #X方向ベクトルは0で固定しており
    #Y方向,Z方向を変更しています.
    def obliquIncidence(self,n):
        BFL = 93.41
        d_slit = 10
        ai = math.atan(d_slit/BFL)
        at = math.asin(ai)
        rs = (math.sin(ai-at)/math.sin(ai+at))**2
        rp = (math.tan(ai-at)/math.tan(ai+at))**2
        Rsp = (rs+rp)/2
        uy = math.sin(at)
        uz = math.cos(at)
        return 0,uy,uz,Rsp
        
#### 組織の親クラス ####        
class Tissue(object): 
    def __init__(self,g,ms,ma,n,d,z_0):
        self.g = g
        self.ms = ms      #scattering coefficient
        self.ma = ma      #absorption coefficient
        self.mt = ma + ms #interaction coefficient
        self.n = n
        self.d = d
        self.z_0 = z_0
        self.z_1 = z_0 + d
    
    def photonMoving(self,x,y,z,ux,uy,uz,s): #光子の移動
        l = s/self.mt
        nx = x + ux*l
        ny = y + uy*l
        nz = z + uz*l
        return nx,ny,nz
    
    def photonAbsorption(self,w): #光子の吸収
        nw = w * (1 - self.ma/self.mt)
        return nw
    
    def photonScattering(self): #光子の散乱
        #r = random.random()
        shita = math.acos((1/(2*self.g))*(1+self.g**2-((1-(self.g)**2)/(1-self.g+2*self.g*random.random()))**2))
        fai = 2*math.pi*random.random()
        return shita, fai
    
    def vectorConv(self,ux,uy,uz): #ベクトルの変更
        s,f = self.photonScattering()
        nux = math.sin(s)*(ux*uz*math.cos(f) - uy*math.sin(f))/(math.sqrt(1-uz**2)) + ux*math.cos(s)
        nuy = math.sin(s)*(uy*uz*math.cos(f) + ux*math.sin(f))/(math.sqrt(1-uz**2)) + uy*math.cos(s)
        nuz = -math.sin(s)*math.cos(f)*math.sqrt(1-uz**2) + uz*math.cos(s)
        return nux, nuy, nuz
        
    def exVectorConv(self,ux,uy,uz): #|uz| > 0.99999の時に発動
        s,f = self.photonScattering()
        nux = math.sin(s)*math.cos(f)
        nuy = math.sin(s)*math.sin(f)
        nuz = uz*math.cos(s)/abs(uz)
        return nux, nuy, nuz
    
    
    def distanceBoundary(self,uz,z): #境界までの距離
        if uz < 0:
            db = (self.z_0 - z)/uz
        else:
            if uz > 0:
                db = (self.z_1 - z)/uz
            else:
                db = 100
        return db
    
    def hittngPotision(self,x,y,z,ux,uy,uz,db): #境界との衝突地点
        x = x + ux*db
        y = y + uy*db
        z = z + uz*db
        return x, y, z
    
    def angleOfIncidence(self,uz): #入射角
        ai = math.acos(abs(uz))
        return ai
    
    def angleOfTrancemission(self,uz,nt): #屈折角
        ai = self.angleOfIncidence(uz)
        at = math.asin((self.n/nt)*math.sin(ai))
        return at
        
    def reflectance(self,uz,nt): #内部反射率
        ai = self.angleOfIncidence(uz)
        if (self.n > nt) and (ai > math.asin(nt/self.n)) :
            Ra = 1
        else:
            if abs(uz) > 0.9999:
                Ra = 0
            else:
                at = self.angleOfTrancemission(uz,nt)
                rs = (math.sin(ai-at)/math.sin(ai+at))**2
                rp = (math.tan(ai-at)/math.tan(ai+at))**2
                Ra = (rs+rp)/2
        return Ra
    
    def newDirectionByRef(self,ux,uy,uz):
        uz_n = -uz
        return ux,uy,uz_n
    
    def newDirectionByTra(self,ux,uy,uz,nt):
        at = self.angleOfTrancemission(uz,nt)
        ux_n = ux*self.n/nt
        uy_n = uy*self.n/nt
        uz_n = uz*math.cos(at)/abs(uz)
        return ux_n,uy_n,uz_n

    
####################　光路解析用クラス　####################

#### スリットの親クラス ####
class Slit(object):
    def __init__(self,outerD,slitD,width,thickness,position):
        self.outerD = outerD
        self.slitD = slitD
        self.width = width
        self.thickness = thickness
        self.position = position
        self.d_out = self.slitD/2 + self.width/2
        self.d_in = self.slitD/2 - self.width/2
        self.position = position
        self.front_z = position + thickness
        self.back_z = position
        
    def hittingPotision(self,x,y,z,ux,uy,uz,hit_z):
        db = (hit_z - z)/uz
        x_slit = x + db*ux
        y_slit = y + db*uy
        z_slit = hit_z
        return x_slit,y_slit,z_slit
            
            
#### レンズの親クラス ####            
class Lens (object):
    def __init__(self,outerD,efl,bfl,ct,et,r,n,position,typ):
        self.outerD = outerD
        self.efl = efl
        self.bfl = bfl
        self.ct = ct
        self.et = et
        self.r = r
        self.n = n
        self.position = position
        if typ == 1:
            self.center = self.position - (self.ct - self.r)
        else:
            if typ == 2:
                self.center = self.position + (self.ct - self.r)
            else:    
                print("レンズの向きが入力されていないか間違っています")
        self.x_l = 0
        self.y_l = 0
        self.z_l = self.center
        
    def hittingPointPlano(self,x,y,z,ux,uy,uz,hit_z):
        db = (hit_z - z)/uz
        x = x + db*ux
        y = y + db*uy
        z = hit_z
        return x,y,z            
            
    def hittingPointConvex(self,x,y,z,ux,uy,uz,typ): #円弧の内側からの衝突の場合 typ = 1,外側からの場合 typ = 2
        delta = ((x-self.x_l)*ux+(y-self.y_l)*uy+(z-self.z_l)*uz)**2-(x-self.x_l)**2-(y-self.y_l)**2-(z-self.z_l)**2+self.r**2
        if typ == 1:
            t = -((x-self.x_l)*ux + (y-self.y_l)*uy + (z-self.z_l)*uz) + math.sqrt(delta)
        if typ == 2:
            t = -((x-self.x_l)*ux + (y-self.y_l)*uy + (z-self.z_l)*uz) - math.sqrt(delta)
        x = x + t*ux
        y = y + t*uy
        z = z + t*uz
        return x,y,z
    
    def VectorConvPlano(self,x,y,z,ux,uy,uz,e,direction):
        nx,ny,nz = self.orthVectorPlano()
        snel_co = self.snelLow(direction)
        ux = snel_co*(ux - (ux*nx + uy*ny + uz*nz)*nx) + nx*math.sqrt(e)
        uy = snel_co*(uy - (ux*nx + uy*ny + uz*nz)*ny) + ny*math.sqrt(e)
        uz = snel_co*(uz - (ux*nx + uy*ny + uz*nz)*nz) + nz*math.sqrt(e)
        return ux,uy,uz
        
    def VectorConvConvex(self,x,y,z,ux,uy,uz,e,typ,direction):
        nx,ny,nz = self.orthVectorConvex(x,y,z,typ)
        snel_co = self.snelLow(direction)
        ux = snel_co*(ux - (ux*nx + uy*ny + uz*nz)*nx) + nx*math.sqrt(e)
        uy = snel_co*(uy - (ux*nx + uy*ny + uz*nz)*ny) + ny*math.sqrt(e)
        uz = snel_co*(uz - (ux*nx + uy*ny + uz*nz)*nz) + nz*math.sqrt(e)
        return ux,uy,uz        
        
    def reflect(self,ux,uy,uz,nx,ny,nz):
        e = 1 - ((1/self.n)**2)*(1 - (ux*nx + uy*ny + uz*nz))
        return e
    
    def orthVectorPlano(self):
        nx = 0
        ny = 0
        nz = -1
        return nx,ny,nz
    
    def orthVectorConvex(self,x,y,z,typ):
        if typ == 1:
            nx = (x-self.x_l)/self.r
            ny = (y-self.y_l)/self.r
            nz = (z-self.z_l)/self.r
        if typ == 2:
            nx = -(x-self.x_l)/self.r
            ny = -(y-self.y_l)/self.r
            nz = -(z-self.z_l)/self.r            
        return nx,ny,nz
    
    def snelLow(self,direction):
        if direction == 1:
            snel_co = 1/self.n
        if direction == 0:
            snel_co = self.n/1
        return snel_co
        
    
##### 曲面が負の方向を向いているレンズ #####        
class Lens1(Lens):
    def __init__(self,outerD,efl,bfl,ct,et,r,n,position):
        super().__init__(outerD,efl,bfl,ct,et,r,n,position,1)
        
    def opticalAnalysis(self,x,y,z,ux,uy,uz):
        x,y,z = self.hittingPointPlano(x,y,z,ux,uy,uz,self.position)
        nx,ny,nz = self.orthVectorPlano()
        e = self.reflect(ux,uy,uz,nx,ny,nz)
        if e <= 0:
            leReflect = True
        else:
            leReflect = False
            ux,uy,uz = self.VectorConvPlano(x,y,z,ux,uy,uz,e,1)
            
            x,y,z = self.hittingPointConvex(x,y,z,ux,uy,uz,1)
            nx,ny,nz = self.orthVectorConvex(x,y,z,1)
            e = self.reflect(ux,uy,uz,nx,ny,nz)
            if e <= 0:
                leReflect = True
            else:
                leReflect = False
                ux,uy,uz = self.VectorConvConvex(x,y,z,ux,uy,uz,e,1,0)
        return x,y,z,ux,uy,uz,leReflect

##### 曲面が正の方向を向いたレンズ #####
class Lens2(Lens): 
    def __init__(self,outerD,efl,bfl,ct,et,r,n,position):
        super().__init__(outerD,efl,bfl,ct,et,r,n,position,2)
        
    def opticalAnalysis(self,x,y,z,ux,uy,uz):
        x,y,z = self.hittingPointConvex(x,y,z,ux,uy,uz,2)
        nx,ny,nz = self.orthVectorConvex(x,y,z,2)
        e = self.reflect(ux,uy,uz,nx,ny,nz)
        if e <= 0:
            leReflect = True
        else:
            leReflect = False
            ux,uy,uz = self.VectorConvConvex(x,y,z,ux,uy,uz,e,2,1)    
            
            x,y,z = self.hittingPointPlano(x,y,z,ux,uy,uz,self.position)
            nx,ny,nz = self.orthVectorPlano()
            e = self.reflect(ux,uy,uz,nx,ny,nz)
            if e <= 0:
                leReflect = True
            else:
                leReflect = False
                ux,uy,uz = self.VectorConvPlano(x,y,z,ux,uy,uz,e,0)
        return x,y,z,ux,uy,uz,leReflect
    
    
#### フォトダイオードのクラス ####
class Photodiode(object):
    def __init__(self,d,position):
        self.d = d
        self.r = d/2
        self.position = position
        self.count = 0
        self.record_w = 0
    
    def hittingPotision(self,x,y,z,ux,uy,uz,hit_z):
        db = (hit_z - z)/uz
        x = x + db*ux
        y = y + db*uy
        z = hit_z
        return x,y,z
    
    def absorbance(self,nPh):
        if self.record_w == 0:
            ab = 0
        else:
            ab = math.log(self.record_w/nPh)
        return ab

In [ ]:
#### 各種関数 ####

#計算時間表示用
def timeManage(rap_time):
    time_s = rap_time
    time_m = 0
    time_h = 0
    if time_s >= 60:
        time_m = int(time_s/60)
        time_s = time_s - 60*time_m
        if time_s >= 60:
            time_h = int(time_m/60)
            time_m = time_m - 60*time_h
    return time_s,time_m,time_h


### モンテカルロに用いる関数たち ###
#現在の層の確認
def cullentLayer(z,uz):
    tissue = bone
    if d_skin == 0:
        if z <= c_bone.z_1:
            tissue = c_bone
        else:
            tissue = bone
    elif z <= skin.z_1:
        if z == skin.d and uz > 0:
            tissue = c_bone
        else:
            tissue = skin
    elif z <= c_bone.z_1 and z >= c_bone.z_0:
        if z == c_bone and uz > 0:
            tissue = bone
        else:
            tissue = c_bone
    return tissue

#ntの決定
def nextNt(tissue,uz):
    if tissue == c_bone:
        if uz > 0:
            nt = bone.n
        else:
            nt = skin.n
    elif tissue == bone:
        nt = c_bone.n
    elif tissue == skin:
        if uz > 0:
            nt = c_bone.n
        else:
            nt = 1
    elif d_skin == 0:
        if tissue == c_bone:
            if uz > 0:
                nt = bone.n
            else:
                nt = 1
        else:
            nt = c_bone
    return nt
    
#モンテカルロ法の関数
def monteCalroMeth(timer_start,fname):
    
    number = nPh / 10
    list_sample = np.zeros((nPh, 7)).astype(np.float32)
    flag_1 = False
    count = 0
    rap_time = 0
    n_count = 0
    list_x = []
    list_z = []
    rd = 0

    while count < 10:

        sub_number = 0

        while number > sub_number:
            sub_number += 1
            ph = Photon()
            ux,uy,uz,Rsp = ph.nomalIncidence(n_skin)
            if d_skin == 0:
                ux,uy,uz,Rsp = ph.nomalIncidence(n_bone)
            ph.w = ph.w - Rsp
            x = 0; y = 0; z = 0
            z_bottom = 0

            while 1:

                s = -math.log(random.random())

                if (z >= bone.z_1 ): #海綿骨を通過した時
                    print("out of tissue")
                    flag_1 = True
                    break

                tissue = cullentLayer(z,uz)#光子の存在する層の確認

                db = tissue.distanceBoundary(uz, z)

                if db * tissue.mt <= s:
                    s = s - db*tissue.mt
                    x,y,z = tissue.hittngPotision(x,y,z,ux,uy,uz,db)

                    nt = nextNt(tissue,uz) #ntの決定

                    Ra = tissue.reflectance(uz,nt)

                    if random.random() <= Ra:

                        ux,uy,uz = tissue.newDirectionByRef(ux,uy,uz)
                    else:
                        ux,uy,uz = tissue.newDirectionByTra(ux,uy,uz,nt)
                        if(z <= 0):
                            flag_1 = False
                            rd = rd + ph.w
                            break

                else:

                    x,y,z = tissue.photonMoving(x,y,z,ux,uy,uz,s)

                    if z <= 0:#例外処理
                        db = abs((z-tissue.z_0)/uz)
                        x,y,z = tissue.hittngPotision(x,y,z,db)

                        nt = 1
                        Ra = tissue.reflectance(uz,nt) 
                        if random.random() <= Ra:

                            ux,uy,uz = tissue.newDirectionByRef(ux,uy,uz)
                            continue
                        else:
                            ux,uy,uz =tissue.newDirectionByTra(ux,uy,uz,nt)

                            flag_1 = False
                            break

                    #list_x = list_x + [x] 
                    #list_z = list_z + [z]

                    if z_bottom < z:
                        z_bottom = z

                    s = 0

                    ph.w = tissue.photonAbsorption(ph.w)

                    if abs(uz) <= 0.99999:
                        ux,uy,uz = tissue.vectorConv(ux,uy,uz)

                    else:
                        ux,uy,uz = tissue.exVectorConv(ux,uy,uz)

                    ud = math.sqrt(ux**2 + uy**2 + uz**2)
                    ux = ux/ud; uy = uy/ud; uz = uz/ud

                if ph.w <= ph._wMin:
                    flag_1 = True
                    break
            if flag_1:
                continue

            list_sample[n_count] = [x,y,ux,uy,uz,ph.w,z_bottom]
            n_count += 1



        rap_time = time.time() - timer_start
        time_s,time_m,time_h = timeManage(rap_time)
        par = (count+1)*10
        print(str(par) + " % : "+str(time_h)+"h/"+str(time_m)+"m/"+str(round(time_s,3))+"s")
        count += 1

    #結果の出力　(.csv)
    output_data = pa.DataFrame(list_sample)
    output_data.columns = ['x','y','ux','uy','uz','ph.w','z_bottom']
    output_data.to_csv(fname,index=False)
    #print(output_data)
    print("File name: %s" %fname)

    elapsed_time = time.time() - timer_start
    time_s,time_m,time_h = timeManage(elapsed_time)
    print("Calculation time: "+str(time_h)+"h/"+str(time_m)+"m/"+str(round(time_s,3))+"s")
    #print("Calculation END")
    

### 光路解析に用いる関数たち ###
#光路解析の関数
def opticalAnalysisMeth(timer_start,fname1,fname2):
    
    #平凸レンズでの光路計算
    step = int((end-start)/split) #総ステップ数

    #結果保存用リスト
    output_data_list = np.zeros((step+1,3)).astype(np.float32)

    #Zの位置をlistとして設定
    for i in range (step+1):
        list_distance = list_distance + [start + split*i]

    leReflect = False #レンズ境界面での反射(Trueだと反射、Falseだと透過)
    count = 0
    #Zのステップごとのループ
    while count <= step:
        out_count = 0

        #それぞれの位置関係
        z_record = (start + split*count)*0.1  #z_recordは一回の移動距離を表す
        z_lens1 = -bfl_1 + z_record           #レンズは平面部分を基準に位置を規定
        z_lens2 = z_lens1 - ct_1 -5.5 - ct_2   #レンズ１からコバ厚と６ｃｍ分離した
        z_slit1 = z_lens1                     #スリットの位置はそれぞれレンズの位置と負方向の面を基準として規定
        z_slit2 = z_lens2 + ct_2
        z_pd = z_lens2 - bfl_2

        #レンズとスリット、フォトダイオードのオブジェクトをそれぞれ生成
        lens_1 = Lens1(outerD_1,efl_1,bfl_1,ct_1,et_1,r_1,n_1,z_lens1)
        lens_2 = Lens2(outerD_2,efl_2,bfl_2,ct_2,et_2,r_2,n_2,z_lens2)
        slit_1 = Slit(slit_outerD,slitD,width,thickness,z_slit1)
        slit_2 = Slit(slit_outerD,slitD,width,thickness,z_slit2)
        pd = Photodiode(d_pd,z_pd)

        try:
            chunk = 50000    #chunk行づつファイルの読み込みを行う（ファイルが大きすぎるため一気に行うと危険）
            csv_obj = pa.read_csv(fname1, chunksize = chunk) #CSVファイルの読み込み
            a = np.zeros((chunk,7))                         #読み込んだ後の保存用リスト
            #ファイル読み込み開始
            for r in csv_obj:
                data = list(r.values.flatten())
                for j in range(int(len(data)/7)):
                    if data[7*j] == 0: #エラー箇所の読み飛ばし
                        continue
                    for k in range(7):
                        a[j][k] = data[j*7 + k]
                #フレームデータとして読み込んだdataをlistに直す    
                data = list(map(list, a))
                data = [[float(w) for w in v] for v in data]

                for i in range (len(a)):
                    if data[i][0] == 0:
                        continue
                    x = data[i][0]; y = data[i][1]; z = 0              #試料表面での光子の位置
                    ux = data[i][2]; uy = data[i][3]; uz = data[i][4]  #試料表面でのベクトル
                    w = data[i][5]; z_b = data[i][6]                   #資料表面での光量、光子最深部到達点

                    if(x!=x)or(y!=y)or(ux!=ux)or(uy!=uy)or(uz!=uz)or(w!=w)or(z_b!=z_b):#エラー（虚数）の除去
                        continue

                    if uz == 0:
                        continue
                    out_count = out_count + 1

                    #スリット１の計算
                    x,y,z = slit_1.hittingPotision(x,y,z,ux,uy,uz,slit_1.front_z)
                    dist = math.sqrt(x**2 + y**2)
                    if (dist <= slit_1.d_in) or (dist >= slit_1.d_out):
                        continue

                    x,y,z = slit_1.hittingPotision(x,y,z,ux,uy,uz,slit_1.back_z)
                    dist = math.sqrt(x**2 + y**2)        
                    if (dist <= slit_1.d_in) or (dist >= slit_1.d_out):
                        continue

                    #レンズ１での計算
                    x,y,z,ux,uy,uz,leReflect = lens_1.opticalAnalysis(x,y,z,ux,uy,uz)
                    if leReflect:
                        continue

                    #スリット２の計算
                    x,y,z = slit_2.hittingPotision(x,y,z,ux,uy,uz,slit_2.front_z)
                    dist = math.sqrt(x**2 + y**2)
                    if (dist <= slit_2.d_in) or (dist >= slit_2.d_out):
                        continue

                    x,y,z = slit_2.hittingPotision(x,y,z,ux,uy,uz,slit_2.back_z)
                    dist = math.sqrt(x**2 + y**2)        
                    if (dist <= slit_2.d_in) or (dist >= slit_2.d_out):
                        continue

                    #レンズ２の計算
                    x,y,z,ux,uy,uz,leReflect = lens_2.opticalAnalysis(x,y,z,ux,uy,uz)
                    if leReflect:
                        continue

                    #フォトダイオードの計算
                    x,y,z = slit_2.hittingPotision(x,y,z,ux,uy,uz,pd.position)
                    dist = math.sqrt(x**2 + y**2)

                    if (dist >= pd.r) or (y > 2) or (y < -1):
                        continue
                    #PDで計測された光子の保存
                    pd.count = pd.count + 1
                    pd.record_w = pd.record_w + w

        finally:
            pass

        #まとめ
        output_data_list[count] = [list_distance[count],pd.absorbance(nPh), pd.count, pd.record_w]

        rap_time = time.time() - timer_start
        time_s,time_m,time_h = timeManage(rap_time)
        par = (count+1)*100/(step+1)
        print(str(par) + " % : " + str(time_h)+"h/"+str(time_m)+"m/"+str(round(time_s,3))+"s")
        count += 1

    #結果の出力　(.csv)
    output_data = pa.DataFrame(output_data_list)
    output_data.columns = ['Z','abs','photonNum','W']
    output_data.to_csv(fname2,index=False)
    #print(output_data)
    print("File name: %s" %fname2)

    elapsed_time = time.time() - timer_start
    time_s,time_m,time_h = timeManage(elapsed_time)
    print("Calculation time: "+str(time_h)+"h/"+str(time_m)+"m/"+str(round(time_s,3))+"s")


In [14]:
####        メインプログラム        ####

timer_start = time.time()

#セーブポイントのデータを取得
data = pa.read_csv("automatic_save.csv", header=None)
print(data)
aut_save = np.array([data.ix[0,0],data.ix[1,0]])

optAnaNum = aut_save[1]


### モンテカルロ法の過程 ###
if optAnaNum == 0:
    skinNum = int(aut_save[0]/len(ms_bone))
    msNum = aut_save[0]- skinNum*len(ms_bone)
    
    while skinNum < len(d_skin):
        #皮膚、皮質骨、海綿骨の位置関係
        z_skin = 0
        z_cbone = d_skin[skinNum]
        z_bone = d_skin[skinNum] + d_cbone

        skin = Tissue(g_skin,ms_skin,ma_skin,n_skin,d_skin[skinNum],z_skin)               #皮膚を定義
        c_bone = Tissue(g_cbone,ms_cbone,ma_cbone,n_cbone,d_cbone,z_cbone)    #皮質骨を定義
        
        while msNum < len(ms_bone):
            bone = Tissue(g_bone,ms_bone[msNum],ma_bone,n_bone,d_bone,z_bone) #海綿骨を定義
            
            fname = "nPh"+str(nPh )+"_skin"+str(d_skin[skinNum])+"_ms"+str(ms_bone[msNum]) +".csv"#計算結果の保存ファイルの名前の定義
            
            monteCalroMeth(timer_start, fname) #モンテカルロ計算開始
            
            msNum += 1
            aut_save[0] = skinNum*len(ms_bone) + msNum
            pa.DataFrame(aut_save).to_csv("automatic_save.csv",index=False,header=None)
            print("")
            step_end = "####### MonteCalro step" + str(int(100*(msNum+skinNum*len(ms_bone))/(len(ms_bone)*len(d_skin)))) + "% #######"
            print(step_end)
            print("")
        msNum = 0
        skinNum +=1
else:
    pass

### 光路計算の過程 ###


    0
0  30
1   0
0h/0m/0.071s
10 %
0h/0m/0.072s
20 %
0h/0m/0.076s
30 %
0h/0m/0.08s
40 %
0h/0m/0.081s
50 %
0h/0m/0.082s
60 %
0h/0m/0.082s
70 %
0h/0m/0.086s
80 %
0h/0m/0.091s
90 %
0h/0m/0.161s
100 %
File name: nPh10_skin0.2_ms350.csv
Calculation time: 0h/0m/0.163s
Calculation END

####### MonteCalro step96% #######

0h/0m/0.169s
10 %
0h/0m/0.172s
20 %
0h/0m/0.176s
30 %
0h/0m/0.18s
40 %
0h/0m/0.181s
50 %
0h/0m/0.188s
60 %
0h/0m/0.2s
70 %
0h/0m/0.228s
80 %
0h/0m/0.255s
90 %
0h/0m/0.256s
100 %
File name: nPh10_skin0.2_ms400.csv
Calculation time: 0h/0m/0.257s
Calculation END

####### MonteCalro step100% #######



In [13]:
#セーブポイントのリセット用コード
data = pa.read_csv("automatic_save.csv", header=None)
print(data)
aut_save = np.array([data.ix[0,0],data.ix[1,0]])
print(aut_save)
aut_save[0] = 30
aut_save[1] = 0
print(aut_save)
pa.DataFrame(aut_save).to_csv("automatic_save.csv",index=False,header=None)

    0
0  32
1   0
[32  0]
[30  0]
